In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

class ChessDataset:
    def __init__(self, data_dir, image_size=(256, 256), batch_size=32):
        self.data_dir = data_dir
        self.image_size = image_size
        self.batch_size = batch_size
        self.datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            validation_split=0.2
        )

    def load_data(self):
        train = self.datagen.flow_from_directory(
            self.data_dir,
            subset='training',
            seed=42,
            target_size=self.image_size,
            batch_size=self.batch_size,
            class_mode='categorical'
        )
        validation = self.datagen.flow_from_directory(
            self.data_dir,
            subset='validation',
            seed=42,
            target_size=self.image_size,
            batch_size=self.batch_size,
            class_mode='categorical'
        )
        return train, validation


class ChessModel:
    def __init__(self, input_shape, num_classes):
        self.model = tf.keras.Sequential([
            Conv2D(32, (3, 3), activation="relu", input_shape=input_shape, kernel_regularizer=tf.keras.regularizers.l2(0.0005)),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(64, (3, 3), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.0005)),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(128, (3, 3), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.0005)),
            MaxPooling2D(pool_size=(2, 2)),
            Dropout(0.25),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(num_classes, activation='softmax')
        ])

    def compile_model(self):
        self.model.compile(
            loss="categorical_crossentropy",
            optimizer="adam",
            metrics=["accuracy"]
        )

    def train_model(self, train_data, validation_data, epochs=20):
        history = self.model.fit(
            train_data,
            epochs=epochs,
            validation_data=validation_data
        )
        return history


class ChessTrainer:
    @staticmethod
    def plot_acc_loss(history, epochs):
        # Plot training & validation accuracy values
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Accuracy over ' + str(epochs) + ' Epochs', size=15)
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.show()

        # Plot training & validation loss values
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Loss over ' + str(epochs) + ' Epochs', size=15)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.show()


# Usage
data_dir = 'D:\Datasets\Chess Dataset\data'
image_size = (256, 256)

# Create Dataset
dataset = ChessDataset(data_dir=data_dir, image_size=image_size)
train_data, validation_data = dataset.load_data()

# Create Model
num_classes = len(train_data.class_indices)
input_shape = image_size + (3,)
model = ChessModel(input_shape=input_shape, num_classes=num_classes)
model.compile_model()

# Train Model
trainer = ChessTrainer()
history = trainer.train_model(train_data=train_data, validation_data=validation_data, epochs=20)

# Plot Accuracies and Losses
trainer.plot_acc_loss(history, epochs=20)
